In [46]:
import pandas as pd
import os
import glob
from datetime import datetime as dt
import copy
import numpy as np
import plotly.express as px
myPath = os.getcwd() # current working directory - change this to the location of your exported google data

In [ ]:
def extractData(myDict,keys):
    """
    Reads a nested dictionary.
    
    Args:
        myDict (dict of dict of...): Nested dictionary of arbitrary number of nestings, e.g. like what you get in a json file.
        
        keys (list of strs): List of keys corresponding to the nested dictionaries you want to exract.
        
    Returns:
        element (object): myDict[key0][key1]...[keyn]
    
    Created on 21/03/2021 by Bennett Schneider
    
    """
    subDict = copy.deepcopy(myDict)

    try:
        for key in keys:
            subDict = subDict[key] # recursively pull out keys until you can't anymore
        element = subDict # return the bottom element
    except KeyError:
        element = np.nan # if the key doesn't exist, return none
    except TypeError:
        element = subDict # if you go too far and you've found a non-dict element, return this

    return element

In [2]:
# Rename the unzipped folder you download from google to json. 
# Inside, it should have Takeout/Location History/Semantic Location History
tPath = os.path.join(myPath,r'json','Takeout','Location History','Semantic Location History')

In [3]:
# Get list of json files representing your semantic location. Grouped by year. One file per month
myFiles = []
for subdir, dirs, files in os.walk(os.path.join(tPath)):
    for filename in files:
        myFiles.append(os.path.join(subdir,filename))

In [10]:
# Read all the data into a pandas data and concatenate vertically
myData = []
for file in myFiles:
    myData.append(pd.read_json(file))
myData = pd.concat(myData)

In [ ]:
# Pull placeVisit data out of the pandas dataframe

procData = myData.copy() # copy myData so we don't overwite it in the subsequent steps

# Pull data from the placeVisit -> location position
for col in ['name','address','latitudeE7','longitudeE7']:
    procData[col] = procData['timelineObjects'].map(lambda x: extractData(x,['placeVisit','location',col]))
    
procData = procData[~procData['name'].isna()] # remove rows with no name

# Convert E7 format to standard lat/lon
procData['lat'] = procData['latitudeE7'] / 10.**7
procData['lon'] = procData['longitudeE7'] / 10.**7

In [ ]:
# Pull out timestamps under the placeVisit -> duration position
for col in ['startTimestampMs','endTimestampMs']:
    procData[col] = procData['timelineObjects'].map(lambda x: extractData(x,['placeVisit','duration',col]))
    procData[col] = procData[col].astype(float)/1000 # convert str in ms to float in micro seconds
    procData[col.replace('TimestampMs','')] = procData[col].map(lambda x: dt.fromtimestamp(x)) # convert to datetime

In [76]:
# Flag the the data with total monthly duration at each place
procData['Month'] = procData['start'].map(lambda x: float(dt(x.year,x.month,1).strftime('%Y%m'))) # monthly flag as a float
procData['Duration'] = (procData['end'] - procData['start']) # duration at each location
procData['Duration'] = procData['Duration'].map(lambda x: x.total_seconds()) # convert duration from timedelta to seconds

groupedData = procData.copy() # copy procData so we don't overwrite it in the next step

# Aggregate so we get the total duration spent in each place during each calendar month
groupedData = groupedData.groupby(['Month','name','address']).agg(
    {
        'lat':np.mean,
        'lon':np.mean,
        'Duration':np.sum,
        'start':np.min,
        'end':np.max
    }).reset_index()

In [ ]:
# Plot all points on a map
fig = px.scatter_mapbox(
    groupedData,
    lat="lat",
    lon="lon",
    hover_data=["name", "address",'start','end'],
    color='Month',
    size='Duration',
    color_continuous_scale ='Viridis', zoom=3, height=1000,width=1800)
fig.update_layout(mapbox_style="open-street-map") # use open streetmap as the base
fig.write_html(os.path.join(myPath,'locationsHistory.html')) # save the data to myPath, set in the first cell
fig.show() # show the map in the notebook as well